In [1]:
from sklearn.datasets import make_classification

In [2]:
dataset = make_classification(n_samples=10_000, n_features=20, n_classes=2)

In [3]:
x, y = dataset

In [4]:
from sklearn.preprocessing import StandardScaler

In [5]:
ss = StandardScaler()

In [6]:
standardized_x = ss.fit_transform(x)

In [7]:
p = .1

In [8]:
import tensorflow as tf

In [15]:
inputs = tf.keras.layers.Input((20,), name='numeric_inputs')

In [16]:
x = tf.keras.layers.Dropout(p)(inputs)
x = tf.keras.layers.Dense(100, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
x = tf.keras.layers.Dense(20, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
x = tf.keras.layers.Dense(10, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
out = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(x)

In [21]:
model = tf.keras.models.Model(inputs=inputs, outputs=out)
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [22]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
numeric_inputs (InputLayer)  [(None, 20)]              0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               2100      
_________________________________________________________________
batch_normalization_v2_3 (Ba (None, 100)               400       
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 20)                2020      
_________________________________________________________________
batch_normalization_v2_4 (Ba (None, 20)                80  

In [23]:
import numpy as np

def bootstrap_sample_generator(batch_size):
    while True:
        batch_idx = np.random.choice(
            standardized_x.shape[0], batch_size)
        yield ({'numeric_inputs': standardized_x[batch_idx]}, 
               {'output': y[batch_idx]})

In [25]:
batch_size = 32

model.fit_generator(
    bootstrap_sample_generator(batch_size),
    steps_per_epoch=10_000 / batch_size,
    epochs=5,
    max_queue_size=10,
)

Epoch 1/5
313/312 [==============================] - 1s 2ms/step - loss: 0.2852 - accuracy: 0.8886
Epoch 2/5
313/312 [==============================] - 1s 2ms/step - loss: 0.2176 - accuracy: 0.9125
Epoch 3/5
313/312 [==============================] - 1s 2ms/step - loss: 0.2343 - accuracy: 0.9028
Epoch 4/5
313/312 [==============================] - 1s 2ms/step - loss: 0.2105 - accuracy: 0.9113
Epoch 5/5
313/312 [==============================] - 1s 2ms/step - loss: 0.2131 - accuracy: 0.9135
